<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/LEE/Titanic_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')

#데이터 전처
columns = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
train = train[columns + ['Survived']]
test = test[columns]

train['Age'] = train['Age'].fillna(train['Age'].mean())
test['Age'] = test['Age'].fillna(test['Age'].mean())
train['Fare'] = train['Fare'].fillna(train['Fare'].mean())
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

train = train[train['Parch'] <=5]
train = train[train['Fare'] <=300]

train['Sex'] = train['Sex'].apply(lambda x: 0 if x=='female' else 1)
test['Sex'] = test['Sex'].apply(lambda x: 0 if x=='female' else 1)

train = pd.get_dummies(train, columns=['Embarked'], drop_first=True)
test = pd.get_dummies(test, columns=['Embarked'], drop_first=True)

train_x = train.drop(columns=['Survived'])
train_y = train['Survived']

from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_resampled, y_resampled = smote.fit_resample(train_x,list(train_y))
X_resampled['Survived'] = y_resampled
train_dataset = X_resampled

#로지스틱 회귀 모델
import statsmodels.api as sm
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
formula = """
Survived ~ C(Pclass)+ C(Sex) + scale(Age) + scale(SibSp) + scale(Parch) + scale(Fare) + C(Embarked_Q)+ C(Embarked_S)
"""

model = sm.Logit.from_formula(formula, data=train)
result = model.fit()
y_pred = result.predict(val_x)
y_pred = y_pred.apply(lambda x: 1 if x>=0.5 else 0)

print(confusion_matrix(val_y,y_pred))
print(classification_report(val_y,y_pred))

#의사결정나무 모델
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=6, random_state=0)
model.fit(train_dataset.drop(columns='Survived'),train_dataset['Survived'])

y_pred = model.predict(val_x)
print(confusion_matrix(val_y, y_pred))
print(classification_report(val_y, y_pred))

#랜덤포레스트 모델
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=0)
model.fit(train_dataset.drop(columns='Survived'),train_dataset['Survived'])

y_pred = model.predict(val_x)
print(confusion_matrix(val_y, y_pred))
print(classification_report(val_y, y_pred))

#XGBoost 모델
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state = 0)
model.fit(train.drop(columns='Survived'),train['Survived'])
y_pred = model.predict(val_x)

print(confusion_matrix(val_y, y_pred))
print(classification_report(val_y, y_pred))

#모델 선택
my_model = XGBClassifier(n_estimators=200, learning_rate=0.01, max_depth=5, random_state=0)
my_model.fit(train_dataset.drop(columns='Survived'),train_dataset['Survived'])
XGB_pred = my_model.predict(test)

#제출 준비
y_pred = my_model.predict(test)
submission['Survived'] = y_pred
submission.head(10)

#submission 파일 저장하기
submission.to_csv('submission.csv',index=False)

Optimization terminated successfully.
         Current function value: 0.440644
         Iterations 6
[[83 20]
 [31 44]]
              precision    recall  f1-score   support

           0       0.73      0.81      0.76       103
           1       0.69      0.59      0.63        75

    accuracy                           0.71       178
   macro avg       0.71      0.70      0.70       178
weighted avg       0.71      0.71      0.71       178

[[89 14]
 [29 46]]
              precision    recall  f1-score   support

           0       0.75      0.86      0.81       103
           1       0.77      0.61      0.68        75

    accuracy                           0.76       178
   macro avg       0.76      0.74      0.74       178
weighted avg       0.76      0.76      0.75       178

[[86 17]
 [30 45]]
              precision    recall  f1-score   support

           0       0.74      0.83      0.79       103
           1       0.73      0.60      0.66        75

    accuracy           